In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data =pd.read_csv('/content/drive/MyDrive/proton data/train.csv')

In [4]:
data

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [5]:

from string import punctuation

In [6]:
data['question_text']=data['question_text'].apply(lambda x:re.sub("["+punctuation+"]",' ',x)) # removes punctuation

data['question_text']=data['question_text'].apply(lambda x:re.sub("\d",' ',x)) #removes digit

data['question_text']=data['question_text'].apply(lambda x:re.sub("\s+",' ',x)) # removes whitespaces

data['question_text']=data['question_text'].str.lower() # converts all in to lower

data['question_text'][1]

'do you have an adopted dog how would you encourage people to adopt and not shop '

In [7]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
stop_words = set(stopwords.words('english')) 
len(stop_words)
data['question_text'] = data['question_text'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [9]:
data['question_text'][1]

'adopted dog would encourage people adopt shop'

In [10]:
lemmatizer = WordNetLemmatizer()
data['question_text'] = data['question_text'].apply(lambda x: ' '.join(lemmatizer.lemmatize(term) for term in x.split()))


In [11]:
data= data.drop('qid', axis=1 )

In [12]:
data.head()

,question_text,target
0,quebec nationalist see province nation,0
1,adopted dog would encourage people adopt shop,0
2,velocity affect time velocity affect space geo...,0
3,otto von guericke used magdeburg hemisphere,0
4,convert montra helicon mountain bike changing ...,0


In [13]:
#displaying the categorical values

data['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [14]:
#conversion to list and then displaying the list
text = data['question_text'].tolist()
text[:3]

['quebec nationalist see province nation',
 'adopted dog would encourage people adopt shop',
 'velocity affect time velocity affect space geometry']

In [15]:
#storing the values of sentiment column to variable y

y = data['target']

In [16]:
from keras.preprocessing import sequence
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer

In [17]:
#tokenizer to read all the words present in our corpus

token = Tokenizer()
token.fit_on_texts(text)

In [18]:
#declaring the vocab_size

vocab_size  = len(token.word_index) + 1
vocab_size

177755

In [19]:
#conversion to numerical formats

encoded_text = token.texts_to_sequences(text)

In [20]:
#printing the values of encoded texts of top 3 rows

print(encoded_text[:3])

[[5850, 4036, 81, 4234, 568], [3462, 201, 3, 3279, 4, 2793, 1265], [1790, 247, 12, 1790, 247, 338, 4824]]


In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
#'max_length' = 120 means we are considering max 120 words or token only
#padding='post' means that we padding post the sentence(keeping values 0 if the tokens are not there)

max_length = 50
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')

In [23]:
X.shape

(1306122, 50)

In [24]:
!wget https://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip -q glove.42B.300d.zip


--2021-08-13 11:54:28--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2021-08-13 11:54:28--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  5.16MB/s    in 6m 1s   

2021-08-13 12:00:29 (4.97 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]

replace glove.42B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: [y]es
error: 

In [25]:
!rm glove.42B.300d.zip


In [26]:
y

0          0
1          0
2          0
3          0
4          0
          ..
1306117    0
1306118    0
1306119    0
1306120    0
1306121    0
Name: target, Length: 1306122, dtype: int64

In [28]:
#declaring dict to store all the words as keys in the dictionary and their vector representations as values

glove_vectors = dict()

In [29]:
file = open('glove.42B.300d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    #storing the word in the variable
    vectors = np.asarray(values[1: ])
    #storing the vector representation of the respective word in the dictionary
    glove_vectors[word] = vectors
file.close()

In [30]:
#printing length of glove vectors
len(glove_vectors)

164999

In [31]:
keys = glove_vectors.keys()
len(keys)


164999

We have total 1193514 key values pairs in our dictionary of glove vectors

In [32]:
glove_vectors.get('you').shape

(300,)

2. Since we have taken the glove vectors of 300 dimensions, that’s why the word ‘you’ is having 300 values.

In [35]:
word_vector_matrix = np.zeros((vocab_size, 300))

for word, index in token.word_index.items():
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector
   

In [37]:
from sklearn.model_selection import train_test_split


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [46]:
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate
from keras.models import Model
from sklearn.utils import class_weight
from keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from sklearn.metrics import roc_auc_score, accuracy_score

In [49]:
vec_size = 300

model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length, weights = [word_vector_matrix], trainable = False))

model.add(Conv1D(64, 8, activation = 'relu'))
#here 64 is number of filters and 8 is size of filters
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(16, activation='relu'))
 
model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
32654/32654 [==============================] - 286s 9ms/step - loss: 0.1535 - accuracy: 0.9431 - val_loss: 0.1761 - val_accuracy: 0.9494
Epoch 2/10
32654/32654 [==============================] - 284s 9ms/step - loss: 0.1357 - accuracy: 0.9467 - val_loss: 0.1534 - val_accuracy: 0.9399
Epoch 3/10
32654/32654 [==============================] - 284s 9ms/step - loss: 0.1324 - accuracy: 0.9480 - val_loss: 0.1514 - val_accuracy: 0.9427
Epoch 4/10
32654/32654 [==============================] - 284s 9ms/step - loss: 0.1300 - accuracy: 0.9487 - val_loss: 0.1414 - val_accuracy: 0.9459
Epoch 5/10
32654/32654 [==============================] - 287s 9ms/step - loss: 0.1289 - accuracy: 0.9491 - val_loss: 0.1449 - val_accuracy: 0.9404
Epoch 6/10
32654/32654 [==============================] - 289s 9ms/step - loss: 0.1295 - accuracy: 0.9492 - val_loss: 0.1495 - val_accuracy: 0.9384
Epoch 7/10
32654/32654 [==============================] - 289s 9ms/step - loss: 0.1282 - accuracy: 0.9499 - val_

In [53]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 300)           53326500  
_________________________________________________________________
module_wrapper_9 (ModuleWrap (None, 43, 64)            153664    
_________________________________________________________________
module_wrapper_10 (ModuleWra (None, 21, 64)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 21, 64)            0         
_________________________________________________________________
dense_9 (Dense)              (None, 21, 32)            2080      
_________________________________________________________________
dropout_7 (Dropout)          (None, 21, 32)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 21, 16)           

In [58]:
y_pred = model.predict(X_test)

In [59]:
from  sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test,y_pred)

In [60]:
auc

0.9386018041845117

In [ ]:
b